In [1]:
# import necessary libraries
import numpy as np
from scipy import sparse
from numpy.linalg import norm
import time
import warnings
warnings.filterwarnings('ignore')

#### Data Parsing

In [2]:
def parse_data(file_path):
    # Lists to keep track of origin and destination pages
    origin_pages = []
    destination_pages = []
    # Dictionaries to store out-degree and in-degree information
    out_degree = {}
    in_degree = {}

    with open(file_path, 'r') as file:
        for line in file:
            # Parse the origin and destination pages, adjusting index
            from_page = int(line.split()[0]) - 1
            to_page = int(line.split()[1]) - 1

            origin_pages.append(from_page)
            destination_pages.append(to_page)

            # Update out-degree count for origin pages
            if from_page not in out_degree:
                out_degree[from_page] = 1
            else:
                out_degree[from_page] += 1

            # Update in-degree list for destination pages
            if to_page not in in_degree:
                in_degree[to_page] = [from_page]
            else:
                in_degree[to_page].append(from_page)
            
    # Calculate number of edges and nodes
    total_edges = len(origin_pages)
    total_nodes = len(set(origin_pages) | set(destination_pages))

    for node in range(total_nodes):
        if node not in in_degree:
            in_degree[node] = []

    # Initialize sparse matrix for PageRank calculation
    D_matrix = sparse.lil_matrix((total_nodes, 1))
    for key in out_degree:
        D_matrix[key] = 1 / out_degree[key]

    # Construct the adjacency matrix using sparse representation
    adjacency_matrix = sparse.csr_matrix(
        ([1] * total_edges, (destination_pages, origin_pages)),
        shape=(total_nodes, total_nodes)
    )

    return adjacency_matrix, D_matrix, in_degree

adj_matrix, D_matrix, incoming_links_dict = parse_data("stanweb.dat")

#### 1a) Power method (a=0.85)

In [3]:
def compute_pagerank(G, alpha=0.85, tol=1e-8):

    # Number of nodes (assuming G is a square matrix)
    num_nodes = G.shape[0]
    
    # Compute out-degree ratios scaled by alpha
    out_degree_ratios = G.sum(axis=0).T / alpha
    
    # Initialize rank vector
    rank_vector = np.ones((num_nodes, 1)) / num_nodes
    error = float('inf')
    iterations = 0

    while error > tol:
        iterations += 1
        
        # Compute new rank values
        new_rank_vector = G.dot(rank_vector / out_degree_ratios)
        
        # Calculate leakage due to dead-ends
        leakage = new_rank_vector.sum()
        
        # Reinsert leaked rank values
        new_rank_vector += (1 - leakage) / num_nodes
        
        # Compute error using the Euclidean norm
        error = np.linalg.norm(rank_vector - new_rank_vector)
        
        # Check for nodes that converged by the second iteration
        if iterations == 2:
            converged_nodes = [
                i for i in range(rank_vector.shape[0])
                if np.linalg.norm(rank_vector[i] - new_rank_vector[i]) < tol
            ]
        
        # Update rank vector
        rank_vector = new_rank_vector
    
    return rank_vector, iterations, converged_nodes

import time

# Measure the start time
start_time = time.time()

# Execute the PageRank algorithm
page_ranks, iterations, converged_nodes = compute_pagerank(adj_matrix)

# Measure and print the elapsed time
elapsed_time = time.time() - start_time
print(f"Pagerank (Power Method) elapsed time: {elapsed_time} seconds")

# Print the number of iterations
print("Number of iterations:", iterations)

# Convert the PageRank vector to a flat array
flattened_pr_vector = np.asarray(page_ranks).ravel()

Pagerank (Power Method) elapsed time: 2.489877462387085 seconds
Number of iterations: 72


#### 1a) Gauss Seidel method (a=0.85)

In [4]:
def compute_pagerank_gs(tol, G, D, incoming_links, alpha):

    # Initialize the rank vector with a uniform distribution
    rank_old = np.ones(G.shape[0]) / G.shape[0]
    iteration = 0
    error = 1
    uniform_prob = 1 / G.shape[0]
    D *= alpha
    rank = rank_old.copy()

    # Gauss-Seidel method for PageRank calculation
    while error > tol:
        iteration += 1
        for i in range(G.shape[0]):
            sum_before = 0
            sum_after = 0
            for j in incoming_links[i]:
                if j < i:
                    sum_before += D.data[j][0] * rank[j]
                if j > i:
                    sum_after += D.data[j][0] * rank_old[j]

            rank[i] = uniform_prob + sum_before + sum_after

        # Calculate relative error
        error = np.linalg.norm(rank - rank_old) / np.linalg.norm(rank)
        if iteration == 2:
            # Identify nodes that converged by the second iteration
            converged_nodes = [
                i for i in range(rank.shape[0])
                if np.linalg.norm(rank[i] - rank_old[i]) < tol
            ]

        #print(f"Iteration: {iteration} ==> Relative Error: {error}")
        rank_old = rank.copy()

    return rank, iteration, converged_nodes

# Measure the start time
start_time = time.time()

# Execute the PageRank algorithm using Gauss-Seidel method
gs_ranks, iterations, converged_nodes_gs = compute_pagerank_gs(1e-8, adj_matrix, D_matrix, incoming_links_dict, 0.85)

# Measure and print the elapsed time
elapsed_time_gs = time.time() - start_time
print(f"Pagerank (Gauss Seidel) elapsed time: {elapsed_time_gs} seconds")

# Print the number of iterations
print("Number of iterations:", iterations)

# Convert the Gauss-Seidel PageRank vector to a flat array
flattened_gs_vector = np.asarray(gs_ranks).ravel()



Pagerank (Gauss Seidel) elapsed time: 70.83803915977478 seconds
Number of iterations: 56


In [5]:
# Sort the PageRank vectors and get the indices in descending order
sorted_indices_pr = flattened_pr_vector.argsort()[-len(flattened_pr_vector):][::-1]
sorted_indices_prgs = flattened_gs_vector.argsort()[-len(flattened_gs_vector):][::-1]

# Print the number of differing ranks between the two methods
difference_count = np.sum(sorted_indices_pr != sorted_indices_prgs)
print(f"Power method and Gauss Seidel method difference (a=0.85): {difference_count} ranks.")



Power method and Gauss Seidel method difference (a=0.85): 121652 ranks.


#### 1b) Power/Gauss Seidel methods (a=0.99)

In [6]:
# Measure the start time
start_time = time.time()
# Execute the PageRank algorithm with alpha=0.99 using Google's algorithm
pr_99, iterations_99, converged_nodes_99 = compute_pagerank(adj_matrix, alpha=0.99, tol=1e-8)
# Measure and print the elapsed time
elapsed_time_99 = time.time() - start_time
print(f"Pagerank (Power Method) elapsed time: {elapsed_time_99} seconds")
print("Number of iterations:", iterations_99)
# Convert the PageRank vector to a flat array
flattened_pr_vector_99 = np.asarray(pr_99).ravel()

# Measure the start time
start_time = time.time()
# Execute the PageRank algorithm with alpha=0.99 using the Gauss-Seidel method
gs_ranks_99, iterations_99, converged_nodes_gs_99 = compute_pagerank_gs(1e-8, adj_matrix, D_matrix, incoming_links_dict, 0.99)
# Measure and print the elapsed time
elapsed_time_gs_99 = time.time() - start_time
print(f"\nPagerank (Gauss Seidel) elapsed time: {elapsed_time_gs_99} seconds")
print("Number of iterations:", iterations_99)
# Convert the Gauss-Seidel PageRank vector to a flat array
flattened_gs_vector_99 = np.asarray(gs_ranks_99).ravel()

Pagerank (Power Method) elapsed time: 14.6840181350708 seconds
Number of iterations: 1141

Pagerank (Gauss Seidel) elapsed time: 66.76844573020935 seconds
Number of iterations: 53


In [7]:
# Sort the PageRank vectors for alpha=0.99 and get the indices in descending order
sorted_indices_pr_99 = flattened_pr_vector_99.argsort()[-len(flattened_pr_vector):][::-1]
sorted_indices_prgs_99 = flattened_gs_vector_99.argsort()[-len(flattened_pr_vector):][::-1]

# Print the number of differing ranks between the two methods for alpha=0.99
difference_count_99 = np.sum(sorted_indices_prgs_99 != sorted_indices_pr_99)
print(f"Power method and Gauss Seidel method difference (a=0.99) {difference_count_99} ranks.")

# Print the number of differences in the top 50 ranks for the power method
top_50_difference_power_method = np.sum(sorted_indices_pr[:50] != sorted_indices_pr_99[:50])
print(f"{top_50_difference_power_method} different ranks (power method).")

# Print the number of differences in the top 50 ranks for the Gauss-Seidel method
top_50_difference_gauss_seidel = np.sum(sorted_indices_prgs[:50] != sorted_indices_prgs_99[:50])
print(f"{top_50_difference_gauss_seidel} different ranks (Gauss-Seidel).\n")

Power method and Gauss Seidel method difference (a=0.99) 281887 ranks.
49 different ranks (power method).
33 different ranks (Gauss-Seidel).



#### 1c) Convergence

In [8]:
# Print the number of components that converged at the second iteration using the power method
print(f"Power method: {len(converged_nodes)} components of π converge at the second iteration.")
# Count how many of the converged components are in the top 1000 ranked nodes
fast_converged_count_power = sum(1 for node in converged_nodes if node in sorted_indices_pr[:1000])
print(f"Power method: {fast_converged_count_power} components correspond to the top 1000 ranked nodes.")
# Print the number of components that converged at the second iteration using the Gauss-Seidel method
print(f"\nGauss Seidel method: {len(converged_nodes_gs)} components of π converge at the second iteration.")
# Count how many of the converged components are in the top 1000 ranked nodes
fast_converged_count_gauss_seidel = sum(1 for node in converged_nodes_gs if node in sorted_indices_prgs[:1000])
print(f"Gauss Seidel method: {fast_converged_count_gauss_seidel} components correspond to the top 1000 ranked nodes.")

Power method: 42904 components of π converge at the second iteration.
Power method: 0 components correspond to the top 1000 ranked nodes.

Gauss Seidel method: 34367 components of π converge at the second iteration.
Gauss Seidel method: 1 components correspond to the top 1000 ranked nodes.


#### 2a) Page X (no in-links, out-links)

In [9]:
def parse_connectivity_data(file_path):

    # Lists to store the origin and destination pages
    origin_pages = []
    destination_pages = []

    with open(file_path, 'r') as file:
        # Read and parse each line of the data file
        for line in file.readlines():
            # Adjust indices to be zero-based
            origin_page = int(line.split()[0]) - 1
            destination_page = int(line.split()[1]) - 1

            # Append pages to respective lists
            origin_pages.append(origin_page)
            destination_pages.append(destination_page)
            
    # Calculate number of edges and nodes
    total_edges = len(origin_pages)
    total_nodes = len(set(origin_pages) | set(destination_pages))
    
    # Create the sparse adjacency matrix with an additional dangling node
    adjacency_matrix = sparse.csr_matrix(
        ([1] * total_edges, (destination_pages, origin_pages)),
        shape=(total_nodes + 1, total_nodes + 1)
    )

    return adjacency_matrix

# Load the data using the rephrased function
adj_matrix1 = parse_connectivity_data("stanweb.dat")

# Run the PageRank algorithm using Google's method
page_ranks1, iterations1, converged_nodes1 = compute_pagerank(adj_matrix1)

# Calculate and print the PageRank of the new page X
page_rank_X = np.asarray(page_ranks1[adj_matrix1.shape[0] - 1]).ravel()[0]
print(f"Pagerank of page X: {page_rank_X}")

# Calculate the change in PageRank of the other pages due to the addition of the new page
norm_difference = np.linalg.norm(
    np.asarray(page_ranks[:adj_matrix.shape[0]]).ravel() - np.asarray(page_ranks1[:adj_matrix1.shape[0] - 1]).ravel()
)
print(f"Change in Pagerank of the other pages: {norm_difference}")

# Convert PageRank vectors to flat arrays
flattened_pr_vector = np.asarray(page_ranks).ravel()
flattened_pr_vector1 = np.asarray(page_ranks1[:adj_matrix1.shape[0] - 1]).ravel()

# Sort the PageRank vectors and get the indices in descending order
sorted_indices_pr = flattened_pr_vector.argsort()[-len(flattened_pr_vector):][::-1]
sorted_indices_pr1 = flattened_pr_vector1.argsort()[-len(flattened_pr_vector1):][::-1]

# Print the number of differing ranks between the two sets of PageRank values
rank_changes = np.sum(sorted_indices_pr != sorted_indices_pr1)
print(f"Changes in: {rank_changes} ranks.")

Pagerank of page X: 5.333658781475117e-07
Change in Pagerank of the other pages: 1.2048086354849138e-08
Changes in: 50570 ranks.


#### 2b) Page X with in-links from Page Y

In [10]:
def parse_data_with_additional_edges(file_path):

    # Lists to store the origin and destination pages
    from_pages = []
    to_pages = []

    with open(file_path, 'r') as file:
        # Read and parse each line of the data file
        for line in file.readlines():
            # Adjust indices to be zero-based
            from_page = int(line.split()[0]) - 1
            to_page = int(line.split()[1]) - 1

            # Append pages to respective lists
            from_pages.append(from_page)
            to_pages.append(to_page)
            
    # Calculate number of edges and nodes
    edges = len(from_pages)
    nodes = len(set(from_pages) | set(to_pages))
    
    # Add a new edge Y -> X
    from_pages.append(nodes)
    to_pages.append(nodes + 1)
    
    # Increase the number of nodes by two for X and Y
    nodes += 2
    
    # Increase the number of edges by one
    edges += 1
    
    # Create the sparse adjacency matrix
    adj_matrix2 = sparse.csr_matrix(
        ([1] * edges, (to_pages, from_pages)),
        shape=(nodes, nodes)
    )

    return adj_matrix2

# Load the data using the rephrased function
adj_matrix2 = parse_data_with_additional_edges("stanweb.dat")

# Run the PageRank algorithm using Google's method
page_ranks2, iterations2, converged_nodes2 = compute_pagerank(adj_matrix2)

# Calculate and print the PageRank of the new pages X and Y
page_rank_Xb = np.asarray(page_ranks2[adj_matrix2.shape[0] - 1]).ravel()[0]
print(f"Pagerank of page X: {page_rank_Xb}")
page_rank_Yb = np.asarray(page_ranks2[adj_matrix2.shape[0] - 2]).ravel()[0]
print(f"Pagerank of page Y: {page_rank_Yb}")

# Calculate the change in PageRank of the other pages due to the addition of the new pages
norm_difference = np.linalg.norm(
    np.asarray(page_ranks[:adj_matrix.shape[0]]).ravel() - np.asarray(page_ranks2[:adj_matrix2.shape[0] - 2]).ravel()
)
print(f"Change in Pagerank of the other pages: {norm_difference}")

# Convert PageRank vectors to flat arrays
flattened_page_ranks = np.asarray(page_ranks).ravel()
flattened_page_ranks2 = np.asarray(page_ranks2[:adj_matrix2.shape[0] - 2]).ravel()

# Sort the PageRank vectors and get the indices in descending order
sorted_indices_page_ranks = flattened_page_ranks.argsort()[-len(flattened_page_ranks):][::-1]
sorted_indices_page_ranks2 = flattened_page_ranks2.argsort()[-len(flattened_page_ranks2):][::-1]

# Print the number of differing ranks between the two sets of PageRank values
rank_changes = np.sum(sorted_indices_page_ranks != sorted_indices_page_ranks2)
print(f"Changes in: {rank_changes} ranks.")


Pagerank of page X: 9.867259009702931e-07
Pagerank of page Y: 5.333653518615822e-07
Change in Pagerank of the other pages: 3.4337013674636576e-08
Changes in: 51477 ranks.


#### 2c) Page X with in-links from Page Y and Page Z

In [11]:
def parse_data_with_multiple_additional_edges(file_path):

    # Lists to store the origin and destination pages
    from_pages = []
    to_pages = []

    with open(file_path, 'r') as file:
        # Read and parse each line of the data file
        for line in file.readlines():
            # Adjust indices to be zero-based
            from_page = int(line.split()[0]) - 1
            to_page = int(line.split()[1]) - 1

            # Append pages to respective lists
            from_pages.append(from_page)
            to_pages.append(to_page)
            
    # Calculate number of edges and nodes
    edges = len(from_pages)
    nodes = len(set(from_pages) | set(to_pages))
    
    # Add a new edge Y -> X
    from_pages.append(nodes)
    to_pages.append(nodes + 2)
    
    # Add a new edge Z -> X
    from_pages.append(nodes + 1)
    to_pages.append(nodes + 2)

    # Increase the number of nodes and edges
    nodes += 3
    edges += 2
    
    # Create the sparse adjacency matrix
    adj_matrix3 = sparse.csr_matrix(
        ([1] * edges, (to_pages, from_pages)),
        shape=(nodes, nodes)
    )

    return adj_matrix3

# Load the data using the rephrased function
adj_matrix3 = parse_data_with_multiple_additional_edges("stanweb.dat")

# Run the PageRank algorithm using Google's method
page_ranks3, iterations3, converged_nodes3 = compute_pagerank(adj_matrix3)

# Calculate and print the PageRank of the new pages X, Y, and Z
page_rank_Xc = np.asarray(page_ranks3[adj_matrix3.shape[0] - 1]).ravel()[0]
print(f"Pagerank of page X: {page_rank_Xc}")
page_rank_Yc = np.asarray(page_ranks3[adj_matrix3.shape[0] - 2]).ravel()[0]
print(f"Pagerank of page Y: {page_rank_Yc}")
page_rank_Zc = np.asarray(page_ranks3[adj_matrix3.shape[0] - 3]).ravel()[0]
print(f"Pagerank of page Z: {page_rank_Zc}")

# Calculate the change in PageRank of the other pages due to the addition of the new pages
norm_difference = np.linalg.norm(
    np.asarray(page_ranks[:adj_matrix.shape[0]]).ravel() - np.asarray(page_ranks3[:adj_matrix3.shape[0] - 3]).ravel()
)
print(f"Change in Pagerank of the other pages: {norm_difference}")

# Convert PageRank vectors to flat arrays
flattened_page_ranks = np.asarray(page_ranks).ravel()
flattened_page_ranks3 = np.asarray(page_ranks3[:adj_matrix3.shape[0] - 3]).ravel()

# Sort the PageRank vectors and get the indices in descending order
sorted_indices_page_ranks = flattened_page_ranks.argsort()[-len(flattened_page_ranks):][::-1]
sorted_indices_page_ranks3 = flattened_page_ranks3.argsort()[-len(flattened_page_ranks3):][::-1]

# Print the number of differing ranks between the two sets of PageRank values
rank_changes = np.sum(sorted_indices_page_ranks != sorted_indices_page_ranks3)
print(f"Changes in: {rank_changes} ranks.")


Pagerank of page X: 1.4400850291097994e-06
Pagerank of page Y: 5.333648255766965e-07
Pagerank of page Z: 5.333648255766965e-07
Change in Pagerank of the other pages: 5.662589740950828e-08
Changes in: 52911 ranks.


#### 2d) Page X with out-links to popular pages

In [12]:
# Define a function to load and parse the initial data
def load_initial_data(data):
    from_pages = []
    to_pages = []
    dict_in = {}

    with open(data, 'r') as f:
        for line in f.readlines():
            from_page = int(line.split()[0]) - 1
            to_page = int(line.split()[1]) - 1
            from_pages.append(from_page)
            to_pages.append(to_page)
            if to_page not in dict_in:
                dict_in[to_page] = [from_page]
            else:
                dict_in[to_page].append(from_page)
                
    edges = len(from_pages)
    nodes = len(set(from_pages) | set(to_pages))
    
    csr_matrix = sparse.csr_matrix(([1] * edges, (to_pages, from_pages)), shape=(nodes, nodes))
    
    return csr_matrix, dict_in

# Load the initial data
adj_matrix, dict_in = load_initial_data("stanweb.dat")

# Run the initial PageRank calculation
page_ranks, iterations, converged_nodes = compute_pagerank(adj_matrix)

# Determine top 100 - most popular pages based on PageRank
indices_page_ranks = np.asarray(page_ranks).ravel().argsort()[-len(page_ranks):][::-1]
top_100_pagerank = indices_page_ranks[:100]

# Determine top 100 - most popular pages based on in-degree
list_degree = np.asarray([len(dict_in[x]) for x in sorted(list(dict_in.keys()))])
indices_degree = list_degree.argsort()[-len(list_degree):][::-1]
top_100_indegree = indices_degree[:100]

def parse_data_with_links_to_top_pages(file_path, top_100):

    from_pages = []
    to_pages = []

    with open(file_path, 'r') as file:
        for line in file.readlines():
            from_page = int(line.split()[0]) - 1
            to_page = int(line.split()[1]) - 1
            from_pages.append(from_page)
            to_pages.append(to_page)
            
    edges = len(from_pages)
    nodes = len(set(from_pages) | set(to_pages))
    
    from_pages.append(nodes)
    to_pages.append(nodes + 2)
    
    from_pages.append(nodes + 1)
    to_pages.append(nodes + 2)
    
    for node in top_100:
        from_pages.append(nodes + 2)
        to_pages.append(node)
    
    nodes += 3
    edges += 102
    
    adj_matrix4 = sparse.csr_matrix(([1] * edges, (to_pages, from_pages)), shape=(nodes, nodes))

    return adj_matrix4

def parse_data_with_links_from_y_to_top_pages(file_path, top_100):
 
    from_pages = []
    to_pages = []

    with open(file_path, 'r') as file:
        for line in file.readlines():
            from_page = int(line.split()[0]) - 1
            to_page = int(line.split()[1]) - 1
            from_pages.append(from_page)
            to_pages.append(to_page)
            
    edges = len(from_pages)
    nodes = len(set(from_pages) | set(to_pages))
    
    from_pages.append(nodes)
    to_pages.append(nodes + 2)
    
    from_pages.append(nodes + 1)
    to_pages.append(nodes + 2)
    
    for node in top_100:
        from_pages.append(nodes + 1)
        to_pages.append(node)

    nodes += 3
    edges += 102
    
    adj_matrix5 = sparse.csr_matrix(([1] * edges, (to_pages, from_pages)), shape=(nodes, nodes))

    return adj_matrix5

# Load the data using the rephrased functions
adj_matrix4 = parse_data_with_links_to_top_pages("stanweb.dat", top_100_pagerank)
page_ranks4, iterations4, converged_nodes4 = compute_pagerank(adj_matrix4)

adj_matrix5 = parse_data_with_links_from_y_to_top_pages("stanweb.dat", top_100_pagerank)
page_ranks5, iterations5, converged_nodes5 = compute_pagerank(adj_matrix5)

# If we add links from X to older, popular pages
page_rank_Xd = np.asarray(page_ranks4[adj_matrix4.shape[0] - 1]).ravel()[0]
print(f"Adding links from X to older, popular pages. Pagerank of page X: {page_rank_Xd}")

# If we add links from Y to older, popular pages
page_rank_Xd2 = np.asarray(page_ranks5[adj_matrix5.shape[0] - 1]).ravel()[0]
print(f"Adding links from Y to older, popular pages. Pagerank of page X: {page_rank_Xd2}")

Adding links from X to older, popular pages. Pagerank of page X: 1.4400732985803644e-06
Adding links from Y to older, popular pages. Pagerank of page X: 9.912111253145325e-07
